In [1]:
import numpy as np
import pandas as pd

ratings_data = pd.read_csv("E:\Datasets\ml-latest-small\\ratings.csv")
ratings_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Datasets\\ml-latest-small\\ratings.csv'

In [10]:
import surprise
import pandas as pd
import urllib
import io
import zipfile


In [13]:
dataset = pd.read_csv('data/ratings.txt', sep=' ', names = ['uid', 'iid', 'rating'])

dataset.head()

,uid,iid,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [16]:
dataset.shape

(35497, 3)

In [18]:
lower_rating = dataset['rating'].min()
upper_rating = dataset['rating'].max()
print(lower_rating)
print(upper_rating)

0.5
4.0


In [19]:
reader = surprise.Reader(rating_scale=(0.5,4.))
data = surprise.Dataset.load_from_df(dataset, reader)

In [20]:
alg = surprise.SVDpp()
output = alg.fit(data.build_full_trainset())

In [21]:
pred = alg.predict(uid='50', iid='52')
score = pred.est
print(score)

3.0028030537791928


In [22]:
# making recommendations
# get a list of all movie ids
iids = dataset['iid'].unique()

# Get a list of iids that uid 50 has rated
iids50 = dataset.loc[dataset['uid'] == 50, 'iid']

# Remove the iids that uid 50 has rated from the list of all movie ids
iids_to_pred = np.setdiff1d(iids, iids50)

In [25]:
testset = [[50, iid, 4.] for iid in iids_to_pred]
predictions = alg.test(testset)
predictions[0]

Prediction(uid=50, iid=14, r_ui=4.0, est=3.062209930953068, details={'was_impossible': False})

In [29]:
data

In [24]:
param_grid = {'lr_all' : [.001, .01], 'reg_all' : [.1, .5]}
ga = surprise.model_selection.GridSearchCV(surprise.SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
ga.fit(data)
print(ga.best_params['rmse'])

{'lr_all': 0.01, 'reg_all': 0.1}


In [27]:
pred_ratings = np.array([pred.est for pred in predictions])
# Find the index of the maximum predicted rating
i_max = pred_ratings.argmax()
# Use this to find the corresponding iid to recommned
iid = iids_to_pred[i_max]

In [28]:
print(iid, pred_ratings[i_max])

126 4.0
